## Trying different clustering methods - Homework 3

Author: Kriti Shrivastava (Spire ID: 31041848)

Sepetember 27th, 2017

Datasource: http://archive.ics.uci.edu/ml/datasets/Wholesale+customers

#### Description: 
Applying various clustering algorithms on the data set. Exploring and comparing these methods using interactive visualizations. Analyzing what is different on each, what their parameters are impacting, what's missing, etc.

###### Bokeh Version used: 0.12.6

    Importing necessary packages

In [34]:
from bokeh.plotting import figure, output_notebook, show
import pandas
import os
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.layouts import row
from bokeh.layouts import column
from bokeh.models import HoverTool, CustomJS,BoxZoomTool, ResetTool, BoxSelectTool, LassoSelectTool
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Select
from collections import defaultdict
from bokeh.palettes import Spectral6, viridis
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, AffinityPropagation
import sklearn.cluster as cluster
import numpy as np

    Reading CSV data to Dataframes

In [35]:
path = os.getcwd()
original_path = path + '\data\Wholesale customers data.csv'

original_data = pandas.read_csv(original_path)

output_notebook()

Loading BokehJS ...

 #### Comparing clutering techniques (KMeans, Agglomerative, Spectral)
        
   ##### Interactions: Linked brushing, Single selection widget (Custom JS), reset

In [36]:
clustered_data = original_data
colours = np.array(['#e6194b','#3cb44b','#ffe119','#0082c8','#f58231','blue','red'])

# KMeans Clustering - k=2-5
kmeans = KMeans(n_clusters=2, random_state=0).fit(original_data)
kmeans3 = KMeans(n_clusters=3, random_state=0).fit(original_data)
kmeans4 = KMeans(n_clusters=4, random_state=0).fit(original_data)
kmeans5 = KMeans(n_clusters=5, random_state=0).fit(original_data)

# Agglomerative Clustering
agglomerative = cluster.AgglomerativeClustering(n_clusters=2,linkage="ward").fit(original_data)
agglomerative3 = cluster.AgglomerativeClustering(n_clusters=3,linkage="ward").fit(original_data)
agglomerative4 = cluster.AgglomerativeClustering(n_clusters=4,linkage="ward").fit(original_data)
agglomerative5 = cluster.AgglomerativeClustering(n_clusters=5,linkage="ward").fit(original_data)

# # Affinity Propagation
# ap = AffinityPropagation(damping=0.5, max_iter=200).fit(original_data)
# s3 = pandas.Series(ap.labels_, name='ap')
# clustered_data['ap'] = list(s3)

# Spectral Clustering 2-5
spectral = SpectralClustering(n_clusters=2).fit(original_data)
spectral3 = SpectralClustering(n_clusters=3).fit(original_data)
spectral4 = SpectralClustering(n_clusters=4).fit(original_data)
spectral5 = SpectralClustering(n_clusters=5).fit(original_data)


clrs = colours[kmeans.labels_]
clrs1 = colours[agglomerative.labels_]
clrs3 = colours[spectral.labels_]

all_colors_data = {
    'k2': colours[kmeans.labels_],
    'k3': colours[kmeans3.labels_],
    'k4': colours[kmeans4.labels_],
    'k5': colours[kmeans5.labels_],
    'sp2': colours[spectral.labels_],
    'sp3': colours[spectral3.labels_],
    'sp4': colours[spectral4.labels_],
    'sp5': colours[spectral5.labels_],
    'ag2' : colours[agglomerative.labels_],
    'ag3' : colours[agglomerative3.labels_],
    'ag4' : colours[agglomerative4.labels_],
    'ag5' : colours[agglomerative5.labels_]
}

data = {
    'Total': list(clustered_data['Total']),
    'Grocery': list(clustered_data['Grocery']),
    'clrs': clrs,
    'clrs1': clrs1,
    'clrs3': clrs3
} 

clustered_source = ColumnDataSource(data=data)
all_clustered_source = ColumnDataSource(data = all_colors_data)

callback =  CustomJS(args=dict(clustered_source=clustered_source, all_clustered_source=all_clustered_source), code="""
        var data = clustered_source.data;  
        var all_data = all_clustered_source.data;
        var selected_k = cb_obj.value;
        if(selected_k=="2"){
            data['clrs'] = all_data['k2'];
            data['clrs3'] = all_data['sp2'];
            data['clrs1'] = all_data['ag2'];
        }
        if(selected_k=="3"){
            data['clrs'] = all_data['k3'];
            data['clrs3'] =all_data['sp3'];
            data['clrs1'] = all_data['ag3'];
        }
        if(selected_k=="4"){
            data['clrs'] = all_data['k4'];
            data['clrs3'] =all_data['sp4'];
            data['clrs1'] = all_data['ag4'];
        }
        if(selected_k=="5"){
            data['clrs'] = all_data['k5'];
            data['clrs3'] =all_data['sp5'];
            data['clrs1'] = all_data['ag5'];
        }
        clrs = data['clrs'];
        clrs1 = data['clrs1'];
        clrs3 = data['clrs3'];
        clustered_source.change.emit();
    """)

plot = figure(plot_width=400, plot_height=400, tools=[BoxSelectTool(),LassoSelectTool(), BoxZoomTool(), ResetTool()], title="KMeans clustering")
plot.circle(x='Total', y='Grocery', source=clustered_source, size=5, color='clrs')
plot.yaxis.axis_label = 'Spendings on Grocery'
plot.xaxis.axis_label = 'Total Spendings'

# p1 = figure(plot_width = 400, plot_height = 400, title='Agglomerative clustering', x_range=p.x_range, y_range=p.y_range)  
# p1.scatter(clustered_data, x='Total', y='Grocery', color='k2agg', 
#             title="Agglomerative clustering", xlabel="Total Spendings", ylabel="Spendings on Grocery",plot_width=400, plot_height=400)
  
p1 = figure(plot_width=400, plot_height=400, tools=[BoxSelectTool(),LassoSelectTool(), BoxZoomTool(), ResetTool()], title="Agglomerative clustering")
p1.circle(x='Total', y='Grocery', source=clustered_source, size=5, color='clrs1')
p1.yaxis.axis_label = 'Spendings on Grocery'
p1.xaxis.axis_label = 'Total Spendings'

p3 = figure(plot_width=400, plot_height=400, tools=[BoxSelectTool(),LassoSelectTool(), BoxZoomTool(), ResetTool()], title="Spectral clustering")
p3.circle(x='Total', y='Grocery', source=clustered_source, size=5, color='clrs3')
p3.yaxis.axis_label = 'Spendings on Grocery'
p3.xaxis.axis_label = 'Total Spendings'

options=['2','3','4','5']
select = Select(title="Select Number of clusters:", value="2", options=options, callback=callback)

layout = column(select, row(plot, p1, p3))
show(layout)

C:\Users\Kriti Shrivastava\Anaconda3\lib\site-packages\sklearn\manifold\spectral_embedding_.py:229: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


#### Analysis

For the purpose of comparison, the data set is clustered using all columns/features and the graphs are plotted across the same x and y axis. The x axis is the column 'Grocery' from the data and the y axis corresponds to the total spending across all products. All the plots are linked through brushing, so points of interest can be selected and examined in all clustering techniques together. Library sklearn was used to implement these clustering algorithms and the parameter requirements mentioned below are with repect to the library specifications.

##### K-Means clustering: 
Parameters required- number of clusters

K-Means is the most common clustering technique and is used in variety of applications. It divides the data points into k clusters such that a metric(ex. Euclidean distance) relative to the centroids of the clusters is minimized.
KMeans is fast and scales well to large number of data points. But, it is not a stable algorithm i.e. each time the method is run, different clusters are formed. A point that belonged to particular cluster in first run can become a part of another cluster in second run. The size of the clusters is assumed to be balanced and hence it usually produces clusters with relatively uniform size. Kmeans is sensitive to outliers. This method works well when the shape of clusters is hyper-spherical but performs poorly when the clusters have irregular shapes.


##### Agglomerative clustering:
Parameteres required- number of clusters, linkage type, distance

Agglomerative clustering is a type of hierarchical clustering where each observation/data point starts in its own cluster, and clusters are successively merged together. The linkage criteria determines the metric used for the merge strategy.
By default(as in the case above), sklearn's agglomerative clustering method uses a variance-minimizing approach called 'ward' as its linkage criteria. In this case, euclidean distance is used as affinity metric to compute the linkage. Therefore, the clusters formed using this technique are almost similar to those formed by kmeans clustering. This can be useful to identify which points occur in the same cluster using both these methods and which ones differ. Iterating both the methods several times with different values of k, can be useful in clearly distinguishing such points. Agglomerative clustering is computation intensive(several merge/split decisions) and is slower as compared to kmeans. This method produced same clusters upon multiple runs as opposed to kmeans which was unstable.

##### Spectral clustering:
Parameters required- number of clusters

In spectral clustering, data points as considered as nodes of a connected graph and clusters are found by partitioning this graph into subgraphs(using spectral decomposition). This method uses similarity measure between points to form clusters. By default(as in the case above), sklearn's spectral clustering creates an affinity matrix using the Gaussian(RBF) kernel of the euclidean distance. Kernel can be selected based on the usecase. Spectral clustering works well for a small number of clusters but the performance detoriates as the number of clusters increases. 
The clusters formed by spectral clustering are interminged with each other, as opposed to the ones generated by kmeans and agglomerative which are visually separable.
In general, Spectral clustering considers connectivity between points instead of geometrical proximity(as in Kmeans). So if the data is not well geometrically separated, but clusters are not connected, spectral clustering may work better than Kmeans.


#### References: 
1. http://www.cis.hut.fi/Opinnot/T-61.6020/2008/spectral_kmeans.pdf
2. http://hdbscan.readthedocs.io/en/latest/comparing_clustering_algorithms.html


